In [0]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATASETS
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE CELL.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote
from urllib.error import HTTPError
from zipfile import ZipFile

CHUNK_SIZE = 40960
DATASET_MAPPING = 'cicids2017:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2203430%2F3681869%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20220524%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20220524T175843Z%26X-Goog-Expires%3D259199%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D21c365a3015971fc91745b0c6e94786b55880eb9bb8a7132c658940e167c28e7f26eabb9681c1849d5851c0ef9492619b464e801e308c31fd23b2828eee3994ebf5123988d08363f1333fb1da167412d010c579f299f5092f684e37fc6397efcaca486992e362f88579987d18d289105430705354e8c6a41d47f8bf0d3c431b6791aba754330545495742c597c637c7a286946e7dcfc97a084c93425d054e2cd05fca68db1ccafb72f9af3d49c8fc0ae94bd2bf48bc8cdc6727b9a6715acd35307a6c15eadb0c6499b0878a84eaf0f0f9dc8678b51dcf1d6b426e7b39153c9a84a52eac22f9feed943751fd6fb3e78ce6bfad86a91592a6a3e4ddc2824048534'
KAGGLE_INPUT_PATH='/home/kaggle/input'
KAGGLE_INPUT_SYMLINK='/kaggle'

os.makedirs(KAGGLE_INPUT_PATH, 777)
os.symlink(KAGGLE_INPUT_PATH, os.path.join('..', 'input'), target_is_directory=True)
os.makedirs(KAGGLE_INPUT_SYMLINK)
os.symlink(KAGGLE_INPUT_PATH, os.path.join(KAGGLE_INPUT_SYMLINK, 'input'), target_is_directory=True)

for dataset_mapping in DATASET_MAPPING.split(','):
    directory, download_url_encoded = dataset_mapping.split(':')
    download_url = unquote(download_url_encoded)
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as zipfileres, NamedTemporaryFile() as tfile:
            total_length = zipfileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes zipped')
            dl = 0
            data = zipfileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = zipfileres.read(CHUNK_SIZE)
            print(f'\nUnzipping {directory}')
            with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue
print('Dataset import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataset = pd.read_csv('../input/cicids2017/cleanDatasetWithoutEncoding.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.columns

In [ ]:
for i,each in enumerate(list(dataset.columns)):
    for j in range(i+1,len(dataset.columns)):
        if each != 'Label' and dataset.columns[j] != 'Label':
            sns.jointplot(x=each,y=dataset.columns[j],data=dataset)
            plt.savefig(f'Scatter plot for {each} with {dataset.columns[j]}', dpi=300)
            plt.close('all')